# AST Visitor Patterns

**Part 3 of the AST Guide Series**

This notebook covers advanced visitor patterns for traversing and analyzing Abstract Syntax Trees (ASTs) in Python. These patterns are essential for building robust static analysis tools.

## Prerequisites

Before working through this notebook, you should be familiar with:
- **Part 1**: AST Core Concepts - Basic AST structure and node types
- **Part 2**: AST Essential Nodes - Function, class, and assignment analysis

## What You'll Learn

- Context stack management for tracking nested scopes
- Building qualified names for precise identification
- Scope resolution challenges and practical solutions
- Advanced visitor traversal patterns

---


In [1]:
# Essential imports for AST analysis
import ast
from typing import List, Dict, Any, Optional

# We'll build on these basic visitor patterns
class BaseVisitor(ast.NodeVisitor):
    """Base visitor with common functionality"""
    def __init__(self):
        self.context_stack = []
        self.current_scope = None

    def enter_scope(self, scope_name: str, node):
        """Enter a new scope (function, class, etc.)"""
        self.context_stack.append((scope_name, node))
        self.current_scope = scope_name

    def exit_scope(self):
        """Exit the current scope"""
        if self.context_stack:
            self.context_stack.pop()
            self.current_scope = self.context_stack[-1][0] if self.context_stack else None


## Visitor Traversal Patterns

### 12. Maintaining Context Stacks

Context stacks are essential for tracking where you are in the code structure. As the visitor traverses nested structures (classes within classes, functions within functions), we need to maintain state about the current context. This is critical for building qualified names and understanding scope. The stack pattern ensures we can handle arbitrary nesting levels and always know our current position in the code hierarchy.

The key principle is: push when entering a context, pop when leaving. This must be done carefully to handle all exit paths, including exceptions. Always use try/finally or ensure your visit method structure guarantees the pop happens.

In [2]:
import ast

code = """
class Outer:
    class Middle:
        class Inner:
            def deep_method(self):
                def local_func():
                    x = 10
                    return x
                return local_func()

    def outer_method(self):
        pass

def module_function():
    def nested_function():
        def deeply_nested():
            pass
        return deeply_nested
    return nested_function
"""

tree = ast.parse(code)
print(ast.dump(tree, indent=2))

Module(
  body=[
    ClassDef(
      name='Outer',
      body=[
        ClassDef(
          name='Middle',
          body=[
            ClassDef(
              name='Inner',
              body=[
                FunctionDef(
                  name='deep_method',
                  args=arguments(
                    args=[
                      arg(arg='self')]),
                  body=[
                    FunctionDef(
                      name='local_func',
                      args=arguments(),
                      body=[
                        Assign(
                          targets=[
                            Name(id='x', ctx=Store())],
                          value=Constant(value=10)),
                        Return(
                          value=Name(id='x', ctx=Load()))]),
                    Return(
                      value=Call(
                        func=Name(id='local_func', ctx=Load())))])])]),
        FunctionDef(
          name='outer_method',
          ar

In [3]:
class ContextTracker(ast.NodeVisitor):
    def __init__(self):
        self.class_stack = []  # Track nested classes
        self.function_stack = []  # Track nested functions
        self.contexts_seen = []  # Record all contexts we've visited

    def get_class_context(self):
        """Get current class qualified name."""
        return ".".join(self.class_stack) if self.class_stack else None

    def get_function_context(self):
        """Get current function qualified name."""
        return ".".join(self.function_stack) if self.function_stack else None

    def get_full_context(self):
        """Get complete context including both class and function."""
        parts = []
        if self.class_stack:
            parts.extend(self.class_stack)
        if self.function_stack:
            parts.extend(self.function_stack)
        return ".".join(parts) if parts else "[module]"

    def visit_ClassDef(self, node):
        # Push class onto stack
        self.class_stack.append(node.name)
        context = self.get_full_context()
        self.contexts_seen.append(("class", context))
        print(f"Entering class: {context}")

        # Visit children
        self.generic_visit(node)

        # Pop class from stack (CRITICAL - must always happen)
        self.class_stack.pop()
        print(f"Leaving class: {node.name}")

    def visit_FunctionDef(self, node):
        # Determine if this is a method or function
        is_method = bool(self.class_stack)

        # Push function onto appropriate stack
        if is_method:
            # For methods, we DON'T add to function_stack in this implementation
            # because we want clean qualified names like Class.method
            context = f"{self.get_class_context()}.{node.name}"
            self.contexts_seen.append(("method", context))
            print(f"Found method: {context}")
        else:
            # For functions, we DO track nesting
            self.function_stack.append(node.name)
            context = self.get_full_context()
            self.contexts_seen.append(("function", context))
            print(f"Entering function: {context}")

        # Visit children
        self.generic_visit(node)

        # Pop if we pushed
        if not is_method:
            self.function_stack.pop()
            print(f"Leaving function: {node.name}")

    # Handle async functions the same way
    visit_AsyncFunctionDef = visit_FunctionDef

    def visit_Lambda(self, node):
        # Lambdas are anonymous functions
        self.function_stack.append("[lambda]")
        print(f"Found lambda in: {self.get_full_context()}")
        self.generic_visit(node)
        self.function_stack.pop()


tracker = ContextTracker()
tracker.visit(tree)

print("\nAll contexts visited:")
for ctx_type, ctx_name in tracker.contexts_seen:
    print(f"  {ctx_type:8} -> {ctx_name}")

Entering class: Outer
Entering class: Outer.Middle
Entering class: Outer.Middle.Inner
Found method: Outer.Middle.Inner.deep_method
Found method: Outer.Middle.Inner.local_func
Leaving class: Inner
Leaving class: Middle
Found method: Outer.outer_method
Leaving class: Outer
Entering function: module_function
Entering function: module_function.nested_function
Entering function: module_function.nested_function.deeply_nested
Leaving function: deeply_nested
Leaving function: nested_function
Leaving function: module_function

All contexts visited:
  class    -> Outer
  class    -> Outer.Middle
  class    -> Outer.Middle.Inner
  method   -> Outer.Middle.Inner.deep_method
  method   -> Outer.Middle.Inner.local_func
  method   -> Outer.outer_method
  function -> module_function
  function -> module_function.nested_function
  function -> module_function.nested_function.deeply_nested


### 13. Building Qualified Names

Qualified names uniquely identify elements in your code. They're built by combining context information with the element's local name. This is crucial for distinguishing between methods with the same name in different classes, or variables with the same name in different scopes. The pattern you choose for building qualified names affects how you track and resolve references later.

In [4]:
import ast

code = """
# Module-level variable
logger = Logger()

class Calculator:
    # Class variable
    default_precision = 2

    def add(self, a, b):
        # Method local variable
        result = a + b
        return result

    class InternalHelper:
        def helper_method(self):
            # Nested class method
            temp = 10
            return temp

def process_data(input_data):
    # Function local variable
    processor = DataProcessor()

    def validate():
        # Nested function variable
        is_valid = True
        return is_valid

    return validate()
"""

tree = ast.parse(code)
print(ast.dump(tree, indent=2))

Module(
  body=[
    Assign(
      targets=[
        Name(id='logger', ctx=Store())],
      value=Call(
        func=Name(id='Logger', ctx=Load()))),
    ClassDef(
      name='Calculator',
      body=[
        Assign(
          targets=[
            Name(id='default_precision', ctx=Store())],
          value=Constant(value=2)),
        FunctionDef(
          name='add',
          args=arguments(
            args=[
              arg(arg='self'),
              arg(arg='a'),
              arg(arg='b')]),
          body=[
            Assign(
              targets=[
                Name(id='result', ctx=Store())],
              value=BinOp(
                left=Name(id='a', ctx=Load()),
                op=Add(),
                right=Name(id='b', ctx=Load()))),
            Return(
              value=Name(id='result', ctx=Load()))]),
        ClassDef(
          name='InternalHelper',
          body=[
            FunctionDef(
              name='helper_method',
              args=arguments(


In [5]:
class QualifiedNameBuilder(ast.NodeVisitor):
    def __init__(self):
        self.class_stack = []
        self.function_stack = []
        self.all_names = []  # Store all qualified names we build

    def build_qualified_name(self, local_name, name_type="entity"):
        """Build a fully qualified name based on current context."""
        # Different strategies for different name types

        if name_type == "method":
            # Methods: ClassName.method_name
            if self.class_stack:
                qualified = ".".join(self.class_stack) + f".{local_name}"
            else:
                qualified = local_name

        elif name_type == "variable":
            # Variables: scope-based naming
            if self.function_stack:
                # Function-scoped variable
                qualified = ".".join(self.function_stack) + f".{local_name}"
            elif self.class_stack:
                # Class-scoped (would be class variable)
                qualified = ".".join(self.class_stack) + f".{local_name}"
            else:
                # Module-scoped
                qualified = f"__module__.{local_name}"

        elif name_type == "class":
            # Nested classes: Outer.Inner
            if self.class_stack:
                qualified = ".".join(self.class_stack) + f".{local_name}"
            else:
                qualified = local_name

        else:
            # Default: just use full context
            context_parts = self.class_stack + self.function_stack
            if context_parts:
                qualified = ".".join(context_parts) + f".{local_name}"
            else:
                qualified = local_name

        return qualified

    def visit_ClassDef(self, node):
        # Build qualified name for the class itself
        class_qname = self.build_qualified_name(node.name, "class")
        self.all_names.append(("class", class_qname))
        print(f"Class: {class_qname}")

        # Push to stack and visit
        self.class_stack.append(node.name)
        self.generic_visit(node)
        self.class_stack.pop()

    def visit_FunctionDef(self, node):
        # Build qualified name based on context
        if self.class_stack:
            # It's a method
            func_qname = self.build_qualified_name(node.name, "method")
            self.all_names.append(("method", func_qname))
            print(f"Method: {func_qname}")
            # Don't push methods onto function stack
            self.generic_visit(node)
        else:
            # It's a function
            func_qname = self.build_qualified_name(node.name, "function")
            self.all_names.append(("function", func_qname))
            print(f"Function: {func_qname}")
            # Do push functions onto stack for nested functions
            self.function_stack.append(node.name)
            self.generic_visit(node)
            self.function_stack.pop()

    def visit_Assign(self, node):
        # Track variable assignments with qualified names
        for target in node.targets:
            if isinstance(target, ast.Name):
                var_qname = self.build_qualified_name(target.id, "variable")
                self.all_names.append(("variable", var_qname))
                print(f"Variable: {var_qname}")

        self.generic_visit(node)

    def visit_AnnAssign(self, node):
        # Track annotated assignments
        if isinstance(node.target, ast.Name):
            var_qname = self.build_qualified_name(node.target.id, "variable")
            self.all_names.append(("variable", var_qname))
            print(f"Annotated variable: {var_qname}")

        self.generic_visit(node)


builder = QualifiedNameBuilder()
builder.visit(tree)

print("\nAll qualified names built:")
for name_type, qname in builder.all_names:
    print(f"  {name_type:10} -> {qname}")

print("\nExample lookups:")
print("  'add' in Calculator context -> 'Calculator.add'")
print("  'result' in Calculator.add context -> 'add.result' (function-scoped)")
print("  'logger' at module level -> '__module__.logger'")

Variable: __module__.logger
Class: Calculator
Variable: Calculator.default_precision
Method: Calculator.add
Variable: Calculator.result
Class: Calculator.InternalHelper
Method: Calculator.InternalHelper.helper_method
Variable: Calculator.InternalHelper.temp
Function: process_data
Variable: process_data.processor
Function: process_data.validate
Variable: process_data.validate.is_valid

All qualified names built:
  variable   -> __module__.logger
  class      -> Calculator
  variable   -> Calculator.default_precision
  method     -> Calculator.add
  variable   -> Calculator.result
  class      -> Calculator.InternalHelper
  method     -> Calculator.InternalHelper.helper_method
  variable   -> Calculator.InternalHelper.temp
  function   -> process_data
  variable   -> process_data.processor
  function   -> process_data.validate
  variable   -> process_data.validate.is_valid

Example lookups:
  'add' in Calculator context -> 'Calculator.add'
  'result' in Calculator.add context -> 'add.res

The qualified naming strategy directly impacts your ability to resolve references later. For variable tracking, you need to map variables like `calc` to their types using scope-aware qualified names like `function_name.calc`, ensuring variables in different functions don't interfere with each other.

### 14. A Common Static Analysis Challenge

A fundamental challenge in static analysis is tracking instance method calls. When someone creates an instance of a class and calls its methods, your AST visitor sees the variable name but doesn't know its type. This is the difference between static analysis (what we're doing) and runtime analysis (which would know the actual types).

The challenge is connecting these two pieces of information: the assignment where we learn the variable's type, and the call where we need to know it. Without this connection, a huge category of function calls - arguably the most common pattern in object-oriented Python - goes uncounted in static analysis tools.

In [6]:
import ast

# Let's trace through what the AST sees:
code = """
class Calculator:
    def add(self, a, b):
        return a + b

def foo():
    # Line 6: Assignment
    calc = Calculator()
    # AST sees: Assign(targets=[Name(id='calc')], value=Call(func=Name(id='Calculator')))
    # We can detect: variable 'calc' is assigned result of calling 'Calculator'

    # Line 7: Method call
    calc.add(1, 2)
    # AST sees: Call(func=Attribute(value=Name(id='calc'), attr='add'))
    # We see: something called 'calc' has method 'add' called on it
    # Challenge: What is 'calc'? We don't know without tracking from line 6!
"""

tree = ast.parse(code)
print("AST structure for the assignment and method call:")
print(ast.dump(tree, indent=2))

AST structure for the assignment and method call:
Module(
  body=[
    ClassDef(
      name='Calculator',
      body=[
        FunctionDef(
          name='add',
          args=arguments(
            args=[
              arg(arg='self'),
              arg(arg='a'),
              arg(arg='b')]),
          body=[
            Return(
              value=BinOp(
                left=Name(id='a', ctx=Load()),
                op=Add(),
                right=Name(id='b', ctx=Load())))])]),
    FunctionDef(
      name='foo',
      args=arguments(),
      body=[
        Assign(
          targets=[
            Name(id='calc', ctx=Store())],
          value=Call(
            func=Name(id='Calculator', ctx=Load()))),
        Expr(
          value=Call(
            func=Attribute(
              value=Name(id='calc', ctx=Load()),
              attr='add',
              ctx=Load()),
            args=[
              Constant(value=1),
              Constant(value=2)]))])])


Let's examine what patterns a basic static analysis implementation can and cannot handle:

In [7]:
# A basic implementation only counts these patterns:
patterns_code = """
def bar():
    # Pattern 1: Direct function calls
    some_function()  # ✓ Basic static analysis can count this

    # Pattern 2: self method calls
    self.method()    # ✓ Can count this (using class context)

    # Pattern 3: Class.static_method calls
    Calculator.static_method()  # ✓ Can count this

    # Pattern 4: Instance method calls (THE CHALLENGE)
    obj = Calculator()
    obj.add(1, 2)    # ✗ Cannot count this without variable tracking!

# Real impact: In typical OO code, pattern 4 is extremely common
# Without solving this, static analysis tools miss many function calls!
"""

patterns_tree = ast.parse(patterns_code)


# Let's find the problematic pattern
class PatternAnalyzer(ast.NodeVisitor):
    def visit_Call(self, node):
        if isinstance(node.func, ast.Attribute):
            if isinstance(node.func.value, ast.Name):
                var_name = node.func.value.id
                method_name = node.func.attr
                print(f"Instance method call found: {var_name}.{method_name}()")
                print(f"  Challenge: What type is '{var_name}'? We don't know!")
        elif isinstance(node.func, ast.Name):
            func_name = node.func.id
            print(f"Direct function call: {func_name}() - We can handle this")
        self.generic_visit(node)


analyzer = PatternAnalyzer()
analyzer.visit(patterns_tree)

Direct function call: some_function() - We can handle this
Instance method call found: self.method()
  Challenge: What type is 'self'? We don't know!
Instance method call found: Calculator.static_method()
  Challenge: What type is 'Calculator'? We don't know!
Direct function call: Calculator() - We can handle this
Instance method call found: obj.add()
  Challenge: What type is 'obj'? We don't know!


### 15. The Solution Approach

The solution is to track variable type information as we traverse the AST. When we see an assignment that we can understand (like `calc = Calculator()`), we record that information. When we later see a method call on that variable, we can resolve its type and properly attribute the call.

The key insight is using scope-qualified names to prevent variable name collisions. Without scope qualification, variables with the same name in different functions would overwrite each other's type information.

In [8]:
# Demonstration of the solution approach:
class ScopeAwareResolver(ast.NodeVisitor):
    def __init__(self):
        self.function_stack = []
        self.scoped_variables = {}  # Maps "scope.varname" -> "TypeName"
        self.resolved_calls = []

    def get_current_scope(self):
        if self.function_stack:
            return ".".join(self.function_stack)
        return "__module__"

    def visit_FunctionDef(self, node):
        # Enter function scope
        self.function_stack.append(node.name)
        print(f"Entering scope: {self.get_current_scope()}")

        # Process function body
        self.generic_visit(node)

        # Exit function scope
        self.function_stack.pop()

    def visit_Assign(self, node):
        # Track assignments like: calc = Calculator()
        if len(node.targets) == 1 and isinstance(node.targets[0], ast.Name):
            var_name = node.targets[0].id

            # Check if value is a constructor call
            if isinstance(node.value, ast.Call):
                if isinstance(node.value.func, ast.Name):
                    class_name = node.value.func.id
                    if class_name[0].isupper():  # Looks like a class
                        scope = self.get_current_scope()
                        qualified_var = f"{scope}.{var_name}"
                        self.scoped_variables[qualified_var] = class_name
                        print(f"Tracked: {qualified_var} = {class_name}")

        self.generic_visit(node)

    def visit_Call(self, node):
        # Resolve calls like: calc.add()
        if isinstance(node.func, ast.Attribute):
            if isinstance(node.func.value, ast.Name):
                var_name = node.func.value.id
                method_name = node.func.attr
                scope = self.get_current_scope()
                qualified_var = f"{scope}.{var_name}"

                # Try to resolve the variable's type
                if qualified_var in self.scoped_variables:
                    class_name = self.scoped_variables[qualified_var]
                    resolved = f"{class_name}.{method_name}"
                    self.resolved_calls.append(resolved)
                    print(f"Resolved call: {var_name}.{method_name}() -> {resolved}()")
                else:
                    print(f"Unresolved call: {var_name}.{method_name}() (unknown type)")

        self.generic_visit(node)


# Test the solution:
test_code = """
def process_data():
    calc = Calculator()
    result = calc.add(10, 20)  # Should resolve to Calculator.add

def analyze_data():
    calc = Analyzer()  # Different type, same variable name!
    calc.analyze()      # Should resolve to Analyzer.analyze
"""

tree = ast.parse(test_code)
resolver = ScopeAwareResolver()
resolver.visit(tree)

print(f"\nScoped variables: {resolver.scoped_variables}")
print(f"Resolved calls: {resolver.resolved_calls}")

Entering scope: process_data
Tracked: process_data.calc = Calculator
Resolved call: calc.add() -> Calculator.add()
Entering scope: analyze_data
Tracked: analyze_data.calc = Analyzer
Resolved call: calc.analyze() -> Analyzer.analyze()

Scoped variables: {'process_data.calc': 'Calculator', 'analyze_data.calc': 'Analyzer'}
Resolved calls: ['Calculator.add', 'Analyzer.analyze']


Let's examine the AST structure of our test code to understand what the resolver is processing:

In [9]:
# Let's see the AST structure of our test code
print("AST structure of test code:")
print(ast.dump(tree, indent=2))

AST structure of test code:
Module(
  body=[
    FunctionDef(
      name='process_data',
      args=arguments(),
      body=[
        Assign(
          targets=[
            Name(id='calc', ctx=Store())],
          value=Call(
            func=Name(id='Calculator', ctx=Load()))),
        Assign(
          targets=[
            Name(id='result', ctx=Store())],
          value=Call(
            func=Attribute(
              value=Name(id='calc', ctx=Load()),
              attr='add',
              ctx=Load()),
            args=[
              Constant(value=10),
              Constant(value=20)]))]),
    FunctionDef(
      name='analyze_data',
      args=arguments(),
      body=[
        Assign(
          targets=[
            Name(id='calc', ctx=Store())],
          value=Call(
            func=Name(id='Analyzer', ctx=Load()))),
        Expr(
          value=Call(
            func=Attribute(
              value=Name(id='calc', ctx=Load()),
              attr='analyze',
              ctx

### 16. Why Scope Matters

Without scope tracking, our variable type dictionary would have collisions whenever the same variable name is used in different contexts. This is extremely common in real code - think of how many functions have variables named `result`, `data`, `temp`, etc. Scope qualification ensures each variable gets a unique identifier based on where it's defined.

In [10]:
# Demonstration of why scope is critical:
collision_code = """
# Module level
processor = DataProcessor()

def foo():
    calc = Calculator()  # Without scope: {"calc": "Calculator"}
    calc.add(1, 2)       # Resolves correctly

def bar():
    calc = Processor()   # Without scope: {"calc": "Processor"} - OVERWRITES!
    calc.process()       # Now foo's calc would also be thought to be Processor!

def baz():
    # Even worse: what if we check variables after all assignments?
    calc = Calculator()
    temp = calc.add(1, 2)
    calc = Processor()   # Same variable reassigned!
    calc.process()       # Which type is calc? It changed mid-function!
"""

print("Demonstrating variable name collisions:")
collision_tree = ast.parse(collision_code)

# Let's run our resolver on this collision-prone code
collision_resolver = ScopeAwareResolver()
collision_resolver.visit(collision_tree)

print("\nWith scope qualification, each 'calc' gets a unique identifier:")
for qualified_name, type_name in collision_resolver.scoped_variables.items():
    print(f"  {qualified_name} -> {type_name}")

print("\nThis prevents the collision problem where variables would overwrite each other!")

Demonstrating variable name collisions:
Tracked: __module__.processor = DataProcessor
Entering scope: foo
Tracked: foo.calc = Calculator
Resolved call: calc.add() -> Calculator.add()
Entering scope: bar
Tracked: bar.calc = Processor
Resolved call: calc.process() -> Processor.process()
Entering scope: baz
Tracked: baz.calc = Calculator
Resolved call: calc.add() -> Calculator.add()
Tracked: baz.calc = Processor
Resolved call: calc.process() -> Processor.process()

With scope qualification, each 'calc' gets a unique identifier:
  __module__.processor -> DataProcessor
  foo.calc -> Calculator
  bar.calc -> Processor
  baz.calc -> Processor

This prevents the collision problem where variables would overwrite each other!


Module scope needs special handling because module-level variables are accessible from all functions:

In [11]:
# Why module scope needs special handling:
module_scope_code = """
# Module-level variables are accessible from all functions
logger = Logger()

def function_one():
    logger.info("Starting")  # Should resolve to Logger.info
    # No local 'logger', so check module scope

def function_two():
    logger = CustomLogger()  # Local variable shadows module variable
    logger.debug("Testing")  # Should resolve to CustomLogger.debug

def function_three():
    global logger
    logger = NewLogger()     # Modifies module-level logger
    logger.warn("Warning")   # Should resolve to NewLogger.warn
"""

module_tree = ast.parse(module_scope_code)
module_resolver = ScopeAwareResolver()
module_resolver.visit(module_tree)

print("Module scope handling:")
for qualified_name, type_name in module_resolver.scoped_variables.items():
    print(f"  {qualified_name} -> {type_name}")

Tracked: __module__.logger = Logger
Entering scope: function_one
Unresolved call: logger.info() (unknown type)
Entering scope: function_two
Tracked: function_two.logger = CustomLogger
Resolved call: logger.debug() -> CustomLogger.debug()
Entering scope: function_three
Tracked: function_three.logger = NewLogger
Resolved call: logger.warn() -> NewLogger.warn()
Module scope handling:
  __module__.logger -> Logger
  function_two.logger -> CustomLogger
  function_three.logger -> NewLogger


In [12]:
# Demonstration of the scope resolution algorithm:
class ScopeResolver(ast.NodeVisitor):
    def __init__(self):
        self.function_stack = []
        self.scoped_variables = {}

    def get_current_scope(self):
        if self.function_stack:
            return ".".join(self.function_stack)
        return "__module__"

    def resolve_variable(self, var_name):
        """Show the resolution process."""
        current_scope = self.get_current_scope()

        # Step 1: Check current function scope
        function_scoped = f"{current_scope}.{var_name}"
        if function_scoped in self.scoped_variables:
            print(f"Found in function scope: {function_scoped}")
            return self.scoped_variables[function_scoped]

        # Step 2: Check module scope
        module_scoped = f"__module__.{var_name}"
        if module_scoped in self.scoped_variables:
            print(f"Found in module scope: {module_scoped}")
            return self.scoped_variables[module_scoped]

        # Step 3: Check if it might be a class name (capitalized)
        if var_name[0].isupper():
            print(f"Treating as class name: {var_name}")
            return var_name

        # Step 4: Give up - unresolvable
        print(f"Cannot resolve: {var_name}")
        return None


# Test the resolution algorithm with some examples
resolver = ScopeResolver()
resolver.scoped_variables = {"__module__.global_var": "GlobalType", "my_function.local_var": "LocalType"}
resolver.function_stack = ["my_function"]

print("Resolution examples:")
print("1. Looking for 'local_var' in function:")
resolver.resolve_variable("local_var")

print("\n2. Looking for 'global_var' in function (falls back to module):")
resolver.resolve_variable("global_var")

print("\n3. Looking for 'ClassName' (treats as class):")
resolver.resolve_variable("ClassName")

print("\n4. Looking for unknown variable:")
resolver.resolve_variable("unknown_var")

Resolution examples:
1. Looking for 'local_var' in function:
Found in function scope: my_function.local_var

2. Looking for 'global_var' in function (falls back to module):
Found in module scope: __module__.global_var

3. Looking for 'ClassName' (treats as class):
Treating as class name: ClassName

4. Looking for unknown variable:
Cannot resolve: unknown_var


---

## Next Steps

You've now mastered the essential visitor patterns for AST analysis! These patterns form the foundation for building sophisticated static analysis tools.

### Continue Your Learning

- **Part 4**: AST Scope Tracking - Advanced scope resolution and variable tracking
- **Part 5**: AST Type Annotations - Deep dive into type annotation analysis
- **Part 6**: AST Practical Techniques - Real-world patterns and debugging

### Key Patterns to Remember

1. **Always maintain context stacks** when traversing nested structures
2. **Build qualified names** for precise element identification
3. **Handle scope boundaries** carefully in your visitor methods
4. **Use generic_visit()** to ensure complete tree traversal

These patterns will serve as the foundation for more advanced AST analysis techniques in the remaining parts of this guide series.
